In [37]:
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
df = pd.read_csv('house_price_bd.csv')

In [3]:
df.shape

(3865, 9)

There are **3865** rows (House Informations)

In [4]:
df['Location'].str.split(',').str.get(1).nunique()

106

 There are **106** unique locations

In [5]:
df['location_area'] = df['Location'].str.split(',').str.get(1).str.lower()

# Location
Replacing   
(1)The (r-a) from basundhara r-a  
(2) The ward numbers from specific wards, such as "9 no. pahartoli ward" becomes only pahartoli ward  
(3) And removing the whitespaces using str.strip()    

In [6]:
df['location_area'] = df['location_area'].str.replace('r-a','').str.replace(r'\b\d+\s*no\.?\s*','',regex=True,flags=re.IGNORECASE).str.strip()

# Price_in_taka
(1) Removing the ৳ from prices   
(2) convert the values in Crores  
(3) rounding up the values to 2

In [7]:
def convert_to_crores(x):
    converted = (x/10000000)
    return converted

In [8]:
df['Price_in_Cr'] = df['Price_in_taka'].str.replace('৳','').str.replace(',','').str.strip().astype(float).apply(lambda x:convert_to_crores(x)).round(2)

In [9]:
df.drop(['Price_in_taka'],axis=1,inplace=True)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3865 entries, 0 to 3864
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Title             3865 non-null   object 
 1   Bedrooms          2864 non-null   float64
 2   Bathrooms         2864 non-null   float64
 3   Floor_no          3181 non-null   object 
 4   Occupancy_status  3766 non-null   object 
 5   Floor_area        3766 non-null   float64
 6   City              3865 non-null   object 
 7   Location          3859 non-null   object 
 8   location_area     3828 non-null   object 
 9   Price_in_Cr       3865 non-null   float64
dtypes: float64(4), object(6)
memory usage: 302.1+ KB


# City
Replacing the "-city" word from narayanganj  
There are **5** cities 

In [11]:
df['City'] = df['City'].str.replace('-city','').str.strip()

In [12]:
df['City'].value_counts()

City
dhaka          1650
chattogram     1470
narayanganj     420
gazipur         180
cumilla         145
Name: count, dtype: int64

# Floor_no

In [15]:
df['Floor_no'].value_counts()

Floor_no
1                       532
4                       418
5                       402
2                       397
3                       366
6                       316
7                       252
8                       228
9                       167
10                       30
11                       26
12                       19
13                        8
14                        2
4th to 8th Backside       2
8th                       2
1st                       1
G+7                       1
0+7                       1
A1,A2,A3,A4,A5,A6,A7      1
1F                        1
18                        1
5th                       1
17                        1
Name: count, dtype: int64

In [13]:
df['Floor_no'].str.strip()

0       3
1       1
2       6
3       4
4       4
       ..
3860    8
3861    7
3862    9
3863    9
3864    8
Name: Floor_no, Length: 3865, dtype: object

Droping where floor_no = "Merin City - Purbach" because they seems to like outliers

In [14]:
df = df[~df['Floor_no'].str.contains('Merin City - Purbach',na=False)]

Removing all the problematic rows with isin() function

In [27]:
rare_floor = df['Floor_no'].value_counts()[df['Floor_no'].value_counts()<=2].index

In [31]:
df = df[~df['Floor_no'].isin(rare_floor)]

In [32]:
df['Floor_no'].value_counts()

Floor_no
1     532
4     418
5     402
2     397
3     366
6     316
7     252
8     228
9     167
10     30
11     26
12     19
13      8
Name: count, dtype: int64

# Occupancy_status
This is the most useless column , might drop it

In [43]:
df['Occupancy_status'].value_counts()

Occupancy_status
vacant      3743
occupied       3
Name: count, dtype: int64

# Bedrooms

In [44]:
df['Bedrooms'].value_counts()

Bedrooms
3.0     2184
2.0      405
4.0      219
7.0        8
16.0       5
5.0        5
8.0        4
1.0        3
6.0        3
18.0       2
33.0       2
22.0       2
50.0       2
23.0       2
21.0       2
13.0       1
20.0       1
27.0       1
15.0       1
Name: count, dtype: int64

# Bathrooms

In [45]:
df['Bathrooms'].value_counts()

Bathrooms
3.0     1758
2.0      651
4.0      323
5.0       65
1.0       22
10.0      21
8.0        8
6.0        4
Name: count, dtype: int64

In [47]:
df.sample(3)

,Title,Bedrooms,Bathrooms,Floor_no,Occupancy_status,Floor_area,City,Location,location_area,Price_in_Cr
1682,This 1616 Sq Ft Apartment Is Up For For Sale I...,3.0,3.0,6,vacant,1616.0,chattogram,"CDA Avenue, Muradpur",muradpur,1.15
2065,A Convenient 1253 Sq Ft Residential Flat Is Pr...,3.0,3.0,7,vacant,1253.0,chattogram,"Dewan Bazar, Bakalia",bakalia,0.56
1382,Nearly Finished 1600 Square Feet Apartment For...,3.0,3.0,8,vacant,1600.0,dhaka,"Sector 15, Uttara",uttara,1.60


In [48]:
df.to_csv('house_price_cleaned.csv',index=False)